In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [25]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import glob

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)

In [17]:
archivos = glob.glob("../data/raw/5.0-poblacion-fonasa/*.csv")
df_fonasa = pd.concat(pd.read_csv(archivo, encoding="latin-1") for archivo in archivos)

df_fonasa["ANO_INFORMACION"] = df_fonasa.MES_INFORMACION.astype(str).str[:4]

df_fonasa["REGION"] = df_fonasa["REGION"].str.upper().str.strip()
df_fonasa["SEXO"] = df_fonasa["SEXO"].str.upper().str.strip()
df_fonasa["SERVICIO_SALUD"] = df_fonasa["SERVICIO_SALUD"].str.upper().str.strip()
df_fonasa["COMUNA"] = df_fonasa["COMUNA"].str.upper().str.strip()

df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].replace(
    {"Más de 99 años": "99 años", "S.I.": "-1"}
)
df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].str.split().str[0].astype(int)

In [20]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in df_pais_fonasa["REGION"].unique():
    df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
    ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO ORIENTE'")
df_ssmc_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO CENTRAL'")

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa

In [29]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "SEXO == 'HOMBRE'", # Hombres
    "mujeres": "SEXO == 'MUJER'", # Mujeres
    "recien_nacidos_vivos": "", # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15", # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20", # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45", # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60", # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65", # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75", # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60", # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75", # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75", # Mayores a 75 anios
}

Finalmente, se calculará la cantidad de población para distinto estratos y grupos etarios.

In [68]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_FONASA, query_strings, "CUENTA_BENEFICIARIOS", "FONASA"
)

# Une todos los resultaods en un unico DataFrame
poblaciones_estratos_calculados = pd.concat(poblaciones_estratos_calculados)
poblaciones_estratos_calculados = poblaciones_estratos_calculados.reset_index(
    names=["Edad Incidencia", "Estrato"]
)